In [4]:
import pandas as pd
import os

os.getcwd()

data = pd.read_csv("/Users/siddharthrekhade/Downloads/signboard_text_dataset.csv")

data["n_label"] = data["label"].map({"advertisement" : 0, "road sign" : 1, "store sign" : 2, "warning" : 3, "informational" : 4})

data.sample(n = 10)

,text,label,n_label
1773,Slippery Floor! Authorized Personnel Only,warning,3
89,Mega Deal - 21% OFF!,advertisement,0
1202,Bakery - Open 24x7,store sign,2
1772,Keep Out! Be Careful,warning,3
1827,Keep Out! Proceed With Caution,warning,3
897,No Parking,road sign,1
1936,Keep Out! Proceed With Caution,warning,3
2275,Welcome to Parking Area,informational,4
1222,Electronics - Open 9-5,store sign,2
47,Mega Deal - 76% OFF!,advertisement,0


In [10]:
from transformers import DistilBertTokenizer

texts = data["text"].tolist()
labels = data["n_label"]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

In [7]:
from transformers import DistilBertModel
import torch

model = DistilBertModel.from_pretrained("distilbert-base-uncased")

with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state[:, 0, :]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [12]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, hidden_dim=768, num_classes=5):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

model = Classifier()

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

X = embeddings
y = torch.tensor(labels)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 8

for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for xb, yb in loader:  # loader = your dataloader
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_preds.extend(preds.argmax(dim=1).tolist())
        all_labels.extend(yb.tolist())

    avg_loss = total_loss / len(loader)
    acc = accuracy_score(all_labels, all_preds)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {acc:.4f}")

Epoch 1/8 - Loss: 0.0005 - Accuracy: 1.0000
Epoch 2/8 - Loss: 0.0001 - Accuracy: 1.0000
Epoch 3/8 - Loss: 0.0001 - Accuracy: 1.0000
Epoch 4/8 - Loss: 0.0000 - Accuracy: 1.0000
Epoch 5/8 - Loss: 0.0000 - Accuracy: 1.0000
Epoch 6/8 - Loss: 0.0000 - Accuracy: 1.0000
Epoch 7/8 - Loss: 0.0000 - Accuracy: 1.0000
Epoch 8/8 - Loss: 0.0000 - Accuracy: 1.0000
